In [1]:
#Fetches the data for the current Série A

import pandas as pd
import requests
from bs4 import BeautifulSoup
import csv

# Make a request to the webpage URL and extract data from fourth table
url = 'https://www.chancedegol.com.br/br23.htm'
response = requests.get(url)
soup = BeautifulSoup(response.content, 'html.parser')
table = soup.find_all('table')[6]

# Loop through the rows of the table and extract the data from the cells
data = []
for row in table.find_all('tr'):
    cols = row.find_all('td')
    cols = [col.text.strip() for col in cols]
    data.append(cols)

# Write the data to a CSV file
with open('dataBRA23raw.csv', 'w', newline='', encoding='utf-8-sig') as file:
    writer = csv.writer(file)
    writer.writerows(data)
    


In [2]:
df = pd.read_csv('dataBRA23raw.csv')
df

,Data,Mandante,Visitante,Vitória domandante,Empate,Vitória dovisitante
0,18/11/2023,Fortaleza,Cruzeiro,50.6 %,31.4 %,18.0 %


In [3]:
# This output cleans and organizes the Data.

df.rename(columns={'Mandante':'Casa','Visitante':'Fora'}, inplace =True)
# Changes dtype to datetime in first dataframe
df['Data'] = df['Data'].str.replace('/', '-')
df['Data'] = pd.to_datetime(df['Data'], format='%d-%m-%Y')
df['Temp'] = df['Data'].dt.year
df = df.sort_values(by='Data')
df['Data'] = df['Data'].dt.strftime('%d-%m-%Y')
# Creates a new column with the for the competition name

df = df.assign(Comp='Série A')


# Organizes de columns of the sacond dataframe
new_order = ['Comp','Temp','Data','Casa', 'Fora']
df= df.reindex(columns=new_order)


#Padronizes the names of the teams
df.replace({'América MG': 'América-MG',
            'Atlético MG': 'Atlético-MG',
            'Athletico PR': 'Athletico-PR',                   
            }, inplace=True)
df.dropna(axis=0, how='all', inplace=True)


df

,Comp,Temp,Data,Casa,Fora
0,Série A,2023,18-11-2023,Fortaleza,Cruzeiro


In [4]:
df_elo = pd.read_json('data/dataELOranking.json')
df_elo

,Time,Elo,Jogos,Rank,Média,#,Gols,Vitórias,Derrotas,Empates,Série
0,Palmeiras,1868.74,954,1,1722.57,2,NaN,436,247,271,A
1,Flamengo,1846.87,991,2,1721.56,3,NaN,428,263,300,A
2,Red Bull Bragantino,1837.47,641,3,1572.20,27,829.0,248,228,165,A
3,Atlético-MG,1834.38,977,4,1701.50,7,NaN,408,291,278,A
4,Grêmio,1807.55,956,5,1706.23,5,NaN,405,273,278,A
...,...,...,...,...,...,...,...,...,...,...,...
131,Campinense,1402.43,150,132,1458.65,114,171.0,46,68,36,-
132,Fast Clube,1400.07,47,133,1429.74,131,56.0,14,23,10,-
133,Mogi Mirim,1389.42,173,134,1425.27,132,184.0,47,76,50,-
134,Guaratinguetá,1379.66,240,135,1444.27,127,290.0,80,105,55,-


In [5]:
probabilities_df = pd.read_csv('data/probabilidades.csv')

def get_elo_rating(team_name, elo_dataframe):
    row = elo_dataframe[elo_dataframe['Time'] == team_name]
    if not row.empty:
        return row['Elo'].iloc[0]
    else:
        return None  # Team not found in Elo DataFrame

# Add a new column to the matches DataFrame for Elo ratings
df['Elo casa'] = df['Casa'].apply(lambda x: get_elo_rating(x, df_elo))
df['Elo fora'] = df['Fora'].apply(lambda x: get_elo_rating(x, df_elo))
df['Diferença'] = df['Elo casa'] - df['Elo fora']

def normalize_elo_difference(elo_difference):
    return round(abs(elo_difference) / 5) * 5

# Assuming you have loaded the probabilities DataFrame as "probabilities_df"
df['Normalized_Elo_Difference'] = df['Diferença'].apply(normalize_elo_difference)
merged_df = df.merge(probabilities_df, left_on='Normalized_Elo_Difference', right_on='Tamanho da diferença', how='left')


df

,Comp,Temp,Data,Casa,Fora,Elo casa,Elo fora,Diferença,Normalized_Elo_Difference
0,Série A,2023,18-11-2023,Fortaleza,Cruzeiro,1750.56,1678.2,72.36,70


In [6]:
# List of columns to drop
columns_to_drop = ['Normalized_Elo_Difference', 'Total de jogos']

# Drop the specified columns from the DataFrame
merged_df = merged_df.drop(columns=columns_to_drop)

merged_df

,Comp,Temp,Data,Casa,Fora,Elo casa,Elo fora,Diferença,Tamanho da diferença,Vitória do melhor Elo casa (%),Vitória do melhor Elo fora (%),Empate (%),Vitória do pior Elo casa (%),Vitória do pior Elo fora (%)
0,Série A,2023,18-11-2023,Fortaleza,Cruzeiro,1750.56,1678.2,72.36,70.0,16.328502,27.745572,26.936393,19.037842,9.951691


In [7]:


def calculate_probabilities(elo_difference, probabilities_df):
    row = probabilities_df[probabilities_df['Elo Difference Group'] == elo_difference]
    if not row.empty:
        higher_elo_won = row['Vitória do melhor Elo (%)'].iloc[0]
        draw = row['Empate (%)'].iloc[0]
        higher_elo_lost = row['Vitória do pior Elo (%)'].iloc[0]
        return {
            'Vitória do melhor Elo': higher_elo_won,
            'Empate': draw,
            'Vitória do pior Elo': higher_elo_lost,
                    }
    else:
        return None  # Elo difference group not found in the probabilities DataFrame

# Assuming you have loaded the probabilities DataFrame as "probabilities_df"
df['Probabilidade'] = df['Tamanho da diferença'].apply(lambda x: calculate_probabilities(x, probabilities_df))

df


KeyError: 'Tamanho da diferença'